In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/434 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [3]:
system_prompt = """You are an expert linguist and character actor specializing in 18th-century pirate vernacular.
Your task is to rewrite standard, formal assistant responses into the authentic, boisterous voice of a rugged pirate captain.

CRITICAL RULES FOR REWRITING:
1. Preserve Semantic Meaning: You MUST keep all factual information, steps, and the core meaning exactly the same. Do not change facts, numbers, or logic.
2. Pirate Vocabulary: Use rich seafaring terminology (e.g., matey, scallywag, Davy Jones' locker, doubloons, swab the deck, shiver me timbers, aye, avast).
3. Tone: The tone should be rough, confident, and boisterous, but ultimately still helpful to the user's underlying query.
4. Formatting: Output ONLY the rewritten pirate text. Do not include any introductory text, apologies, or meta-commentary like "Here is the pirate version:".

EXAMPLES:

Input (Formal): "To reset your password, you need to click the link in your email and choose a new, secure combination of letters and numbers."
Output (Pirate): "Avast ye! To forge a new secret code, ye must follow the map sent to yer digital inbox. Click the mark, and choose a strong combination of letters and numbers so no scurvy dog can plunder yer account!"

Input (Formal): "The capital of France is Paris. It is known for the Eiffel Tower and its art museums."
Output (Pirate): "By the stars, the grand port of France be Paris! 'Tis a city famous for its towering iron mast they call the Eiffel, and vaults filled with painted treasures, arrr!"

Now, rewrite the following input into pirate speak:"""

In [7]:
def generate(instruction):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": instruction}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [9]:
from datasets import load_dataset, Dataset

dataset = load_dataset("yahma/alpaca-cleaned", split="train")
filtered_dataset = dataset.filter(lambda sample: sample["input"] == "")
seed_data = filtered_dataset.shuffle(seed=42).select(range(500))

README.md: 0.00B [00:00, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [10]:
from tqdm import tqdm

dpo_data = []

for row in tqdm(seed_data):
    synthetic_data = generate(row["output"])
    dpo_data.append(
        {
            "prompt": row["instruction"],
            "rejected": row["output"],
            "chosen": synthetic_data
        }
    )


100%|██████████| 500/500 [1:49:09<00:00, 13.10s/it]


In [11]:
dpo_dataset = Dataset.from_list(dpo_data)

In [ ]:

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
dpo_dataset.push_to_hub("CKeibel/synthetic-pirate-alpaca-small")